# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import xlsxwriter
import requests 
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 66866726, 'calculationPrice': 'close', 'change': 1.29, 'changePercent': 0.00738, 'close': 176.08, 'closeSource': 'official', 'closeTime': 1695672000393, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 176.04, 'delayedPriceTime': 1695671931929, 'extendedChange': 0.02, 'extendedChangePercent': 0.00011, 'extendedPrice': 176.1, 'extendedPriceTime': 1695686399294, 'high': 176.97, 'highSource': '15 minute delayed price', 'highTime': 1695671999987, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 176.11, 'iexCloseTime': 1695671999972, 'iexLastUpdated': 1695671999972, 'iexMarketPercent': 0.023730495526148112, 'iexOpen': 174.28, 'iexOpenTime': 1695648600483, 'iexRealtimePrice': 176.11, 'iexRealtimeSize': 303, 'iexVolume': 1095702, 'lastTradeTime': 1695671999972, 'latestPrice': 176.08, 'latestSource': 'Close', 'latestTime': 'September 25, 2023', 'latestUpdate': 1695672000393, 'latestVolume': 46172740, 'low': 174.15, 'lowSource': 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

29.59

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
# Initialize an empty list to store individual DataFrames
dfs = []

for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_url)

    if response.status_code == 200:
        data = response.json()

        for symbol in symbol_string.split(','):
            if symbol in data:
                quote_data = data[symbol]['quote']

                # Create a DataFrame for each symbol
                df = pd.DataFrame([[symbol, quote_data['latestPrice'], quote_data['peRatio'], 'N/A']], columns=my_columns)
                dfs.append(df)

# Concatenate all the DataFrames into a final DataFrame
final_df = pd.concat(dfs, ignore_index=True)

final_df


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,111.94,29.3,N/A
1,AAL,12.91,3.01,N/A
2,AAP,56.92,9.86,N/A
3,AAPL,176.08,29.59,N/A
4,ABBV,154.65,31.76,N/A
...,...,...,...,...
491,YUM,124.97,25.25,N/A
492,ZBH,117.35,48.9,N/A
493,ZBRA,228.15,18.27,N/A
494,ZION,34.58,3.59,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
# Sort by 'Price-to-Earnings Ratio' in descending order
final_df.sort_values('Price-to-Earnings Ratio', ascending=False, inplace=True)

# Filter out entries with P/E ratio greater than 0
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]

# Select the top 50 entries
final_df = final_df.head(50)

# Reset the index and drop the old index 
final_df.reset_index(drop=True, inplace=True)

final_df


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WYNN,90.65,1452.72,N/A
1,LVS,45.34,680.78,N/A
2,WELL,83.19,343.05,N/A
3,TAP,62.94,334.43,N/A
4,KSU,293.59,279.61,N/A
5,VTR,42.45,222.02,N/A
6,STE,225.73,189.69,N/A
7,CRM,206.34,129.77,N/A
8,CLX,132.06,110.05,N/A
9,AMZN,131.25,103.35,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_df.index)
for row in final_df.index:
    final_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[row,'Price'])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WYNN,90.65,1452.72,220
1,LVS,45.34,680.78,441
2,WELL,83.19,343.05,240
3,TAP,62.94,334.43,317
4,KSU,293.59,279.61,68
5,VTR,42.45,222.02,471
6,STE,225.73,189.69,88
7,CRM,206.34,129.77,96
8,CLX,132.06,110.05,151
9,AMZN,131.25,103.35,152


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'

data = requests.get(batch_api_url).json()
# print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ration = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit =data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
# print(ev_to_gross_profit)

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        try:
            # Get the necessary data for calculations
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            
            try:
                ev_to_ebitda = enterprise_value / ebitda
            except (TypeError, ZeroDivisionError):
                ev_to_ebitda = np.NaN
                
            try:
                ev_to_gross_profit = enterprise_value / gross_profit
            except (TypeError, ZeroDivisionError):
                ev_to_gross_profit = np.NaN
                
            # Create a new DataFrame with the data for this symbol
            new_data = pd.DataFrame([[
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            ]], columns=rv_columns)
            
            # Concatenate the new data with the main DataFrame
            rv_dataframe = pd.concat([rv_dataframe, new_data], ignore_index=True)
        except KeyError:
            print(f"Data not found for symbol: {symbol}")

rv_dataframe


Data not found for symbol: ABC
Data not found for symbol: ANTM
Data not found for symbol: DISCA
Data not found for symbol: FBHS
Data not found for symbol: HFC
Data not found for symbol: NLOK
Data not found for symbol: RE
Data not found for symbol: VIAC
Data not found for symbol: WLTW


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,111.94,N/A,29.3,N/A,5.86,N/A,4.66,N/A,20.092713,N/A,9.561243,N/A,N/A
1,AAL,12.91,N/A,3.01,N/A,-1.92,N/A,0.1593,N/A,4.482997,N/A,0.586331,N/A,N/A
2,AAP,56.92,N/A,9.86,N/A,1.26,N/A,0.3053,N/A,6.046417,N/A,1.028618,N/A,N/A
3,AAPL,176.08,N/A,29.59,N/A,45.34,N/A,7.12,N/A,22.423142,N/A,16.662103,N/A,N/A
4,ABBV,154.65,N/A,31.76,N/A,20.95,N/A,4.81,N/A,11.656727,N/A,8.336802,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,124.97,N/A,25.25,N/A,-4.15,N/A,5.01,N/A,19.574813,N/A,8.800269,N/A,N/A
492,ZBH,117.35,N/A,48.9,N/A,1.97,N/A,3.39,N/A,15.697840,N/A,5.775778,N/A,N/A
493,ZBRA,228.15,N/A,18.27,N/A,3.9,N/A,2.12,N/A,13.245211,N/A,5.389109,N/A,N/A
494,ZION,34.58,N/A,3.59,N/A,1.04,N/A,1.08,N/A,2.710166,N/A,1.128378,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABMD,381.020,N/A,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
27,ALXN,182.500,N/A,59.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
38,AON,335.530,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
86,CERN,94.920,N/A,47.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
116,CTL,11.000,N/A,9.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
119,CTXS,103.900,N/A,39.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
133,DISCK,24.420,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
162,ETFC,49.260,N/A,14.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
180,FISV,114.230,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
182,FLIR,57.340,N/A,32.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for col in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[col].fillna(rv_dataframe[col].mean(), inplace = True )

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = score(rv_dataframe[metric],rv_dataframe.loc[row,metric])/100  

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,111.94,N/A,29.30,0.709677,5.86,0.770161,4.6600,0.772177,20.092713,0.814516,9.561243,0.743952,N/A
1,AAL,12.91,N/A,3.01,0.096774,-1.92,0.060484,0.1593,0.012097,4.482997,0.078629,0.586331,0.018145,N/A
2,AAP,56.92,N/A,9.86,0.197581,1.26,0.174395,0.3053,0.040323,6.046417,0.137097,1.028618,0.0625,N/A
3,AAPL,176.08,N/A,29.59,0.72379,45.34,0.975806,7.1200,0.887097,22.423142,0.862903,16.662103,0.951613,N/A
4,ABBV,154.65,N/A,31.76,0.766129,20.95,0.955645,4.8100,0.788306,11.656727,0.389113,8.336802,0.673387,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,124.97,N/A,25.25,0.618952,-4.15,0.056452,5.0100,0.796371,19.574813,0.800403,8.800269,0.699597,N/A
492,ZBH,117.35,N/A,48.90,0.905242,1.97,0.339718,3.3900,0.61996,15.697840,0.645161,5.775778,0.443548,N/A
493,ZBRA,228.15,N/A,18.27,0.457661,3.90,0.647177,2.1200,0.461694,13.245211,0.461694,5.389109,0.383065,N/A
494,ZION,34.58,N/A,3.59,0.09879,1.04,0.139113,1.0800,0.209677,2.710166,0.024194,1.128378,0.076613,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]])
    
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)
rv_dataframe
    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,111.94,N/A,29.30,0.709677,5.86,0.770161,4.6600,0.772177,20.092713,0.814516,9.561243,0.743952,0.762097
1,AAL,12.91,N/A,3.01,0.096774,-1.92,0.060484,0.1593,0.012097,4.482997,0.078629,0.586331,0.018145,0.053226
2,AAP,56.92,N/A,9.86,0.197581,1.26,0.174395,0.3053,0.040323,6.046417,0.137097,1.028618,0.0625,0.122379
3,AAPL,176.08,N/A,29.59,0.72379,45.34,0.975806,7.1200,0.887097,22.423142,0.862903,16.662103,0.951613,0.880242
4,ABBV,154.65,N/A,31.76,0.766129,20.95,0.955645,4.8100,0.788306,11.656727,0.389113,8.336802,0.673387,0.714516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,124.97,N/A,25.25,0.618952,-4.15,0.056452,5.0100,0.796371,19.574813,0.800403,8.800269,0.699597,0.594355
492,ZBH,117.35,N/A,48.90,0.905242,1.97,0.339718,3.3900,0.61996,15.697840,0.645161,5.775778,0.443548,0.590726
493,ZBRA,228.15,N/A,18.27,0.457661,3.90,0.647177,2.1200,0.461694,13.245211,0.461694,5.389109,0.383065,0.482258
494,ZION,34.58,N/A,3.59,0.09879,1.04,0.139113,1.0800,0.209677,2.710166,0.024194,1.128378,0.076613,0.109677


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe = rv_dataframe.sort_values('RV Score', ascending=True).head(50).reset_index(drop=True)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,N/A,0.43,0.092742,0.0456,0.0625,0.0934,0.002016,0.270279,0.018145,0.094878,0.002016,0.035484
1,LNC,24.89,N/A,-1.40,0.084677,0.8652,0.096774,0.2720,0.028226,-1.426673,0.016129,0.272008,0.004032,0.045968
2,WBA,21.34,N/A,-5.56,0.068548,0.8656,0.09879,0.1340,0.010081,-5.745485,0.014113,1.079871,0.070565,0.052419
3,AAL,12.91,N/A,3.01,0.096774,-1.9200,0.060484,0.1593,0.012097,4.482997,0.078629,0.586331,0.018145,0.053226
4,GM,33.06,N/A,4.61,0.104839,0.6256,0.078629,0.2641,0.024194,1.224366,0.020161,0.878730,0.042339,0.054032
5,XRX,15.89,N/A,-9.35,0.050403,0.7471,0.086694,0.3530,0.048387,5.472471,0.106855,1.010633,0.054435,0.069355
6,C,41.02,N/A,6.53,0.129032,0.4227,0.066532,0.6013,0.09879,3.585914,0.040323,0.634431,0.020161,0.070968
7,KSS,20.54,N/A,-21.97,0.03629,0.6099,0.076613,0.1282,0.008065,7.696230,0.199597,1.109121,0.072581,0.078629
8,SYF,31.43,N/A,5.81,0.118952,1.0400,0.139113,0.6460,0.104839,3.620023,0.042339,0.846018,0.038306,0.08871
9,WFC,41.56,N/A,10.39,0.205645,0.4676,0.070565,0.7452,0.131048,3.003775,0.02621,0.781252,0.034274,0.093548


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:2500000


In [20]:
position_size = float(portfolio_size)/len(rv_dataframe.index)            
for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/ rv_dataframe.loc[row,'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.43,0.092742,0.0456,0.0625,0.0934,0.002016,0.270279,0.018145,0.094878,0.002016,0.035484
1,LNC,24.89,2008,-1.40,0.084677,0.8652,0.096774,0.2720,0.028226,-1.426673,0.016129,0.272008,0.004032,0.045968
2,WBA,21.34,2343,-5.56,0.068548,0.8656,0.09879,0.1340,0.010081,-5.745485,0.014113,1.079871,0.070565,0.052419
3,AAL,12.91,3872,3.01,0.096774,-1.9200,0.060484,0.1593,0.012097,4.482997,0.078629,0.586331,0.018145,0.053226
4,GM,33.06,1512,4.61,0.104839,0.6256,0.078629,0.2641,0.024194,1.224366,0.020161,0.878730,0.042339,0.054032
5,XRX,15.89,3146,-9.35,0.050403,0.7471,0.086694,0.3530,0.048387,5.472471,0.106855,1.010633,0.054435,0.069355
6,C,41.02,1218,6.53,0.129032,0.4227,0.066532,0.6013,0.09879,3.585914,0.040323,0.634431,0.020161,0.070968
7,KSS,20.54,2434,-21.97,0.03629,0.6099,0.076613,0.1282,0.008065,7.696230,0.199597,1.109121,0.072581,0.078629
8,SYF,31.43,1590,5.81,0.118952,1.0400,0.139113,0.6460,0.104839,3.620023,0.042339,0.846018,0.038306,0.08871
9,WFC,41.56,1203,10.39,0.205645,0.4676,0.070565,0.7452,0.131048,3.003775,0.02621,0.781252,0.034274,0.093548


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer , sheet_name= 'Value  Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
                    'A' : [ 'Ticker',string_template] ,
                    'B' : ['Price',dollar_template] ,
                    'C' : ['Number of Shares to Buy',integer_template],
                    'D' : ['Price-to-Earnings Ratio',float_template],
                    'E' : ['PE Percentile',percent_template],
                    'F' : ['Price-to-Book Ratio', float_template],
                    'G' : ['PB Percentile', percent_template],
                    'H' : ['Price-to-Sales Ratio', float_template],
                    'I' : ['PS Percentile',percent_template],
                    'J' : ['EV/EBITDA',float_template],
                    'K' : ['EV/EBITDA Percentile',percent_template],
                    'L' : ['EV/GP',float_template],
                    'M' : ['EV/GP Percentile',percent_template],
                    'N' : ['RV Score',percent_template]
                    }
for col in column_formats.keys():
    writer.sheets['Value  Strategy'].set_column(f'{col}:{col}', 25, column_formats[col][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()